# <div style='color:#6f67e0;font-weight:800;'>隔多少步保存模型</div>


In [1]:


from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
import os
from stable_baselines3 import PPO

from stable_baselines3.common.results_plotter import load_results, ts2xy
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback



In [2]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

monitor_dir = r'./monitor_log/'
os.makedirs(monitor_dir,exist_ok=True)
env = Monitor(env,monitor_dir)

env = GrayScaleObservation(env,keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4,channels_order='last')


In [3]:
tensorboard_log = r'./tensorboard_log/'
learning_rate = 1e-6
n_steps = 128
model = PPO("CnnPolicy", env, verbose=1,
            tensorboard_log = tensorboard_log,
            learning_rate = learning_rate,
            n_steps = n_steps)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [4]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq, save_model_dir, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = os.path.join(save_model_dir, 'best_model/')
        self.best_mean_reward = -np.inf

    # def _init_callback(self) -> None:
    def _init_callback(self):
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    # def _on_step(self) -> bool:
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            print('self.n_calls: ',self.n_calls)
            model_path1 = os.path.join(self.save_path, 'model_{}'.format(self.n_calls))
            self.model.save(model_path1)

        return True

In [5]:
save_model_dir = r'F:\\RL_Mario1\\'
callback1 = SaveOnBestTrainingRewardCallback(1000, save_model_dir)


model.learn(total_timesteps=5000,callback=callback1)
# model.save("mario_model")

Logging to ./tensorboard_log/PPO_1
----------------------------
| time/              |     |
|    fps             | 12  |
|    iterations      | 1   |
|    time_elapsed    | 10  |
|    total_timesteps | 128 |
----------------------------


D:\software\e_anaconda\envs\pytorch\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


-------------------------------------------
| time/                   |               |
|    fps                  | 19            |
|    iterations           | 2             |
|    time_elapsed         | 12            |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | 3.3341348e-07 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | -0.00407      |
|    learning_rate        | 1e-06         |
|    loss                 | 202           |
|    n_updates            | 10            |
|    policy_gradient_loss | -3.95e-06     |
|    value_loss           | 432           |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 25            |
|    iterations           | 3             |
|    time_elapsed         | 15  